# Imports

In [1]:
import numpy as np
import torch
from torchvision import transforms as T
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

import pacs
from mixstyle import MixStyle

# Config

## Regarding Dataset

In [2]:
NUM_CLASSES = 7
CLASSES = ["dog", "elephant", "giraffe", "guitar", "horse", "house", "person"]
DOMAINS = {"art_painting", "cartoon", "photo", "sketch"}

## Hyperparameters

In [ ]:
EPOCHS = 25
BATCH_SIZE = 32
LEARNING_RATE = 0.001 # Standard value for Adam: 0.001
REGULARIZATION = 0 # Standard value for Adam: 0
BETAS = (0.9, 0.999) # Standard values for Adam: (0.9, 0.999)
MIX_P = 0.5
MIX_ALPHA = 0.1 # Authors recommend to stick wiht alpha=0.1
MIX_TYPE = 'random' # 'random' or 'crossdomain' (Authors report similar performance)

## Image Normalization

In [4]:
# Values for pretrained ResNet
# image_transform = T.Compose([
#     T.Resize(256),
#     T.CenterCrop(224),
#     T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

## Device

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Device: {device}")

Device: cuda


## Model with MixStyle layers

In [ ]:
class MixStyleResNet(models.ResNet):
    # Adapt ResNet-18 implementation from https://docs.pytorch.org/vision/main/_modules/torchvision/models/resnet.html
    # https://github.com/KaiyangZhou/mixstyle-release/blob/master/reid/models/resnet_ms.py
    def __init__(self):
        super().__init__(models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=NUM_CLASSES)
        self.mixstyle = MixStyle(p=MIX_P, alpha=MIX_ALPHA, mix=MIX_TYPE)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.mixstyle(x)
        x = self.layer2(x)
        x = self.mixstyle(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# Set seed for reproducibility

In [ ]:
seed = 42
torch.manual_seed(42)
if device == torch.device("cuda"):
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Training

In [7]:
writer = SummaryWriter()
%load_ext tensorboard
%tensorboard --logdir ./runs

Reusing TensorBoard on port 6007 (pid 11624), started 12 days, 5:36:23 ago. (Use '!kill 11624' to kill it.)

## Training Loop

In [8]:
class AverageMeter:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
def accuracy(target, output):
    batch_size = target.shape[0]
    _, pred = torch.max(output, dim=-1)
    correct = pred.eq(target).sum()
    return correct.item() / batch_size

In [10]:
def train(epoch, target_domain, data_loader, model, optimizer) -> tuple[float, float]:
    """train one epoch"""
    model.train()
    losses = AverageMeter()
    accs = AverageMeter()

    for i, (data, target) in enumerate(data_loader):
        step = (epoch - 1) * len(data_loader) + i + 1
        data = data.to(device)
        target = target.to(device)

        out = model(data)
        loss = F.cross_entropy(out, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(target, out)
        losses.update(loss.item(), out.shape[0])
        accs.update(acc, out.shape[0])

        writer.add_scalar(f'Loss/Train/target={target_domain}', loss.item(), step)
        writer.add_scalar(f'Accuracy/Train/target={target_domain}', acc, step)

    return losses.avg, accs.avg

## Evaluation

In [11]:
def evaluate(data_loader, model, phase="val") -> tuple[float, float]:
    model.eval()
    losses = AverageMeter()
    accs = AverageMeter()

    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)

            out = model(data)
            loss = F.cross_entropy(out, target)

            acc = accuracy(target, out)
            losses.update(loss.item(), out.shape[0])
            accs.update(acc, out.shape[0])
    
    return losses.avg, accs.avg

## Training Loop

In [ ]:
results = []

for target_domain in tqdm(DOMAINS, desc="Target Domain"):
    model = MixStyleResNet()
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=BETAS, weight_decay=REGULARIZATION)

    img_mean, img_std = pacs.get_nomalization_stats(target_domain)
    print(f"Normalization values excluding domain {target_domain}:\n\tmean: {img_mean}\n\tstd: {img_std}")
    image_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=img_mean, std=img_std)
    ])

    train_loader, test_loader = pacs.get_data_loaders(target_domain, train_batch_size=BATCH_SIZE, transform=image_transform)

    best_acc = 0
    for epoch in tqdm(range(1, EPOCHS + 1), desc=f"Epoch ({target_domain})"):
        train_loss, train_acc = train(epoch, target_domain, train_loader, model, optimizer)
        test_loss, test_acc = evaluate(test_loader, model)

        writer.add_scalar(f"Loss/Test/target={target_domain}", test_loss, epoch)
        writer.add_scalar(f"Accuracy/Test/target={target_domain}", test_acc, epoch)

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), f"models/mixstyle/best_{target_domain}.pt")

    model.load_state_dict(torch.load(f"models/mixstyle/best_{target_domain}.pt"))
    _, acc = evaluate(test_loader, model, phase="final")

    results.append(acc)

avg_acc = np.mean(results)
worst_case_acc = np.min(results)

print(f"Average Accuracy: {avg_acc}\nWorst-case Performance: {worst_case_acc}")

Target Domain:   0%|          | 0/4 [00:00<?, ?it/s]

Normalization values excluding domain cartoon:
	mean: tensor([0.7512, 0.7332, 0.7101])
	std: tensor([0.1835, 0.1886, 0.2020])


ValueError: sampler option is mutually exclusive with shuffle